In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
class Delta():
    def __init__(self, capacity):
        self.capacity = capacity
        self.container = []
    def add(self, img):
        self.container.append(img)
        if len(self.container)>10:
            del self.container[0]
        res = self.container[0]
        
        if len(self.container) == 1:
            return res
        
        for i in range(1,len(self.container)):

            res=cv2.absdiff(res, self.container[i])
        
        return res

In [115]:
pts = np.float32([(580,450), (705,450), (200,650), (1160,650)])
def transform_perspective(img, pts):
    rows,cols,ch = img.shape
    
    pts2 = np.float32([[0,0],[500,0],[0,500],[500,500]])
    M = cv2.getPerspectiveTransform(pts,pts2)
    dst = cv2.warpPerspective(img,M,(500,500))
    return dst


def transform_perspective_back(img, pts):
    rows,cols,_ = img.shape
    
    pts2 = np.float32([[0,0],[500,0],[0,500],[500,500]])
    M = cv2.getPerspectiveTransform(pts,pts2)
#     dst = cv2.warpPerspective(img, M, (1280,720),cv2.INTER_LINEAR, cv2.WARP_INVERSE_MAP, cv2.BORDER_CONSTANT, 0);
    dst = cv2.warpPerspective(img, M, (1280,720),cv2.INTER_LINEAR, cv2.WARP_INVERSE_MAP, cv2.BORDER_TRANSPARENT);
    return dst

In [4]:
def lr_split(lst):
    l = []
    r = []
    for i in lst:
        if i[0]<250:
            l.append(i)

        else:
            r.append(i)

    return l,r

In [233]:
def draw_line(point_lists, point_lists2):
    width, height = 500, 500  # picture's size
    img = np.zeros((height, width, 4), np.uint8) # make the background white
    line_width = 25
    for i in range(len(point_lists)-2):
         # change color or make a color generator for your self
        pts = np.array([point_lists[i], point_lists[i+1]], dtype=np.int32)
        cv2.polylines(img, [pts], False, (255,0,0,255), thickness=line_width, lineType=cv2.LINE_AA)
        
        pts = np.array([point_lists2[i], point_lists2[i+1]], dtype=np.int32)
        cv2.polylines(img, [pts], False, (0,0,255,255), thickness=line_width, lineType=cv2.LINE_AA)
       
    return img
    

def draw_road(l, r):
#     l, r = lr_split(pts)
    t = np.arange(0, 500, 20)
    xdata_1 = np.array(l)[:,0]
    ydata_1 = np.array(l)[:,1]
    z_1 = np.polyfit(ydata_1, xdata_1, 2)
    f_1 = np.poly1d(z_1)

    xdata = np.array(r)[:,0]
    ydata = np.array(r)[:,1]
    z_2 = np.polyfit(ydata, xdata, 2)
    f_2 = np.poly1d(z_2)
    
    l1, r1 = [(f_1(i), i) for i in t], [(f_2(i), i) for i in t]
    infill = np.ones((500,500,4), dtype=np.uint8)
#     print(np.array(l1+r1[::-1]))
#     print(r1+l1[::-1])
#     print(r1)
    return draw_line(l1,r1), cv2.fillPoly(infill, [np.array(l1+r1[::-1],dtype=np.int32)], (0,255,0,255))



In [205]:
def lr_split(lst):
    l = []
    r = []
    for i in lst:
        if i[0]<250:
            l.append(i)

        else:
            r.append(i)

    return l,r

In [268]:
delta = Delta(10)
cap = cv2.VideoCapture("project_video.mp4")
fr = 25
count = 0

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 25.0, (1280,720))

while(cap.isOpened()):
  count+=1
  ret, frame = cap.read()
  if ret == True:
    
    pts = np.float32([(580,450), (705,450), (200,650), (1160,650)])
    res = transform_perspective(frame, pts)
    hls = cv2.cvtColor(res, cv2.COLOR_RGB2HLS).astype(np.float)
    s_channel = hls[:,:,2].astype(np.uint8)
    combined = cv2.bitwise_or(yw_data(res),s_channel)
    
    ret,th1 = cv2.threshold(combined,170,255,cv2.THRESH_BINARY)
    
    delt = delta.add(th1)
    
    kernel = np.ones((4,4),np.uint8)
    dilated = cv2.dilate(delt,kernel,iterations=4)
    
#     cv2.imshow('Original',frame)

    
    
    black = np.zeros((500,500,1), dtype=np.uint8)
    mask = cv2.merge([black,black,black])
#     l,r = 
#     print(l)
#     print(r)
    l,r = point_complement(*pt_2(dilated))
    
#     print("=")
#     if (count==fr):
#         img = dilated
#         break
        
#     for i in l:
# #         if (i[0]):
#             cv2.circle(mask, i, 10, (255,0,0) , -1)
    
#     for i in r:
# #         if (i[0]):
#             cv2.circle(mask, i, 10, (0,0,255) , -1)
    mask, poly = draw_road(l,r)
    # print(points_s(dilated))
#     cv2.imshow('Frame',draw_road(points_s(dilated)))
#     print (mask[4])
#     cv2.imwrite("res.png", mask)
#     break
    back = transform_perspective_back(mask ,pts)
    
    
#     cv2.imshow('Frame2',mask)
    b_channel, g_channel, r_channel = cv2.split(frame)
    alpha = np.zeros(b_channel.shape, dtype=b_channel.dtype)
    

    
    infill_tr = transform_perspective_back(poly ,pts)
    
    fram = cv2.merge([b_channel, g_channel, r_channel, alpha])
#     cv2.imwrite("res.png", back)
#     break
#     cv2.imshow('Frame',)
    fram = cv2.addWeighted(fram, 1, infill_tr, 0.4, 0)
    masked_lines = cv2.bitwise_and(fram, fram ,mask = cv2.bitwise_not(back[:,:,3]))+back
    r,g,b,_ = cv2.split(masked_lines)
    out.write(cv2.merge([r,g,b]))
    cv2.imshow('Frame',masked_lines)
#     print(masked_lines.shape)
    a,b,c = cv2.split(res)
    alpha = np.zeros(a.shape, dtype=a.dtype)
    
    res_4 = cv2.merge([a, b, c, alpha])
    cv2.imshow('Frame2', cv2.bitwise_and(res_4, res_4 ,mask = cv2.bitwise_not(mask[:,:,3]))+mask)
#     break
#     if (count==fr):
#         break
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
cap.release()
out.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [186]:
cap = cv2.VideoCapture("project_video.mp4")
def frame_n(cap, frame):
    cap.set(1, frame)
    ret, frame = cap.read()
    return frame

def yw_data(img):
    # img = transform_perspective(frame_n(cap, 280),pts)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV).astype(np.float)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lower_yellow = np.array([20, 100, 100], dtype = 'uint8')
    upper_yellow = np.array([30, 255, 255], dtype='uint8')
    mask_yellow = cv2.inRange(img_hsv, lower_yellow, upper_yellow)
    mask_white = cv2.inRange(gray_image, 200, 255)
    mask_yw = cv2.bitwise_or(mask_white, mask_yellow)
    mask_yw_image = cv2.bitwise_and(gray_image, mask_yw)
    return mask_yw_image

In [63]:
def points_s(img_cap):
    counter = 0
    res_2=[]
    r_r=[]
    for i in range(10):
        histogram = np.sum(img_cap[counter:counter+50], axis=0)
        histogram[histogram <= 12000] = 0
        histogram[histogram > 12000] = 1
        count = 0
        flag = 0
        r_r=[]
        for i in histogram:
            count+=1
            if i==1 and (flag==0):
                interm = count
                flag = 1
            elif i==0 and flag == 1:
    #             r_r.append(((count-interm)//2, counter+25))
                res_2.append((interm +(count-interm)//2, counter+25))
                flag=0
#                 print((interm +(count-interm)//2, counter+50))
#                 print(res_2)

    #     res_2.append(r_r)     
    #     print(histogram)
#         np.append(res,[histogram])
    #     ax.plot(histogram)
        counter+=50
    return res_2
    # plt.imshow(histogram)
# x=points_s(dilated)
# print(x)

In [106]:
def pt_2(img_cap):
    counter = 0
    l=[]
    r=[]
    for i in range(10):
        histogram = np.sum(img_cap[counter:counter+50], axis=0)
        histogram[histogram <= 12000] = 0
        histogram[histogram > 12000] = 1
        count = 0
        flag = 0
        for i in histogram:
            count+=1
            if i==1 and (flag==0):
                interm = count
                flag = 1
            elif i==0 and flag == 1:
                elem = (interm +(count-interm)//2, counter+25)
                if elem[0]<250:
                    l.append(elem)
                else:
                    r.append(elem)

                flag=0
#         if (len(l)-len(r)>0):
        r+=[(None, counter+25) for i in range(len(l)-len(r))]

#         elif (len(r)-len(l)>0):
        l+=[(None, counter+25) for i in range(len(r)-len(l))]
            
        counter+=50
    return l,r
    # plt.imshow(histogram)
# x=points_s(dilated)
# print(x)

In [95]:
# l=[(9, 75), (27, 125), (44, 175), (57, 225), (69, 275), (80, 325), (86, 375), (89, 425), (91, 475)]
# r=[(373, 75), (385, 125), (428, 275), (439, 325), (456, 475)]

# img = cv2.imread("dil.png")
l, r = lr_split(points_s(img))
print(l)
print(r)
# black = np.zeros((500,500,3), dtype=np.uint8)
# for i in l:
#     cv2.circle(black, i, 10, (255,0,0) , -1)

# for i in r:
#     cv2.circle(black, i, 10, (0,0,255) , -1)
# plt.imshow(black)

def point_complement(l, r):
    ln = 0
    mid = 0
    for i in range(len(l)):
        if (l[i][0]!=None and r[i][0]!=None ):
            width = r[i][0]-l[i][0]
            if width > 100:
                ln+=1
                mid += width
    res = mid//ln
    
    out_l=[]
    out_r=[]
    
    for i in range(len(l)):
        if (l[i][0]==None):
#             out_l.append( (r[i][0]-res, r[i][1]))
            l[i] = (r[i][0]-res, r[i][1])
        elif (r[i][0]==None):
#             out_r.append((l[i][0]+res, l[i][1]))
            r[i] = (l[i][0]+res, l[i][1])
    return l, r
print (point_complement(*pt_2(img)))

[(9, 75), (27, 125), (44, 175), (57, 225), (69, 275), (80, 325), (86, 375), (89, 425), (91, 475)]
[(373, 75), (385, 125), (428, 275), (439, 325), (456, 475)]
([(9, 75), (27, 125), (44, 175), (57, 225), (69, 275), (80, 325), (86, 375), (89, 425), (91, 475)], [(373, 75), (385, 125), (405, 175), (418, 225), (428, 275), (439, 325), (447, 375), (450, 425), (456, 475)])


In [264]:
cap = cv2.VideoCapture(0)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.flip(frame,0)

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()